In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pygrib # import pygrib interface to grib_api
import pandas as pd
import os
import sys
import math
import folium
import csv 
import time

start = time.time()

%matplotlib inline

root = os.getcwd()
root_data = root + "/grib_data"
root_URL = "http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/gpv/original/"

#緯度経度の範囲を選択
east_lon=130.400
west_lon= 130.370
north_lat=33.61
south_lat=33.59

#timestomp like 201808010000 means that it is at 2019/04/09/08:00(UTC timezone)
year = "2019"
month = "06"
day = "15"
day2 = "24"
roottime="000000"

def getgrib(year,month,day,roottime,FH):
    
    file_name = "Z__C_RJTD_"+str(year+month+day+roottime)+"_MSM_GPV_Rjp_Lsurf_FH"+str(FH)+"_grib2.bin"
    file_url = root_URL\
                +str(year + "/" + month + "/" + day +"/")\
                +file_name
    root_file= "grib_data/"+file_name
    if os.path.isfile(root_file) == False:
            command = "wget -P grib_data/ "+ file_url
            os.system(command)

    getgrib_output = pygrib.open(root_file)
    return getgrib_output

def getrootgrib(year,month,day,time,FH):
    file_name = "Z__C_RJTD_"+str(year+month+day+roottime)+"_MSM_GPV_Rjp_Lsurf_FH"+str(FH)+"_grib2.bin"   
    return "grib_data/"+file_name

In [2]:
while int(day) < int(day2):
                     
    if int(day) < 10 :
        day="0"+day
    
    FH="00-15"
    
    grbs = getgrib(year,month,day,roottime,FH)
    latlons = grbs

    #check parameters in gribfile
    grb_1 = grbs.select(name='Pressure reduced to MSL')[0]

    # lats,lonsは二次元配列で緯度経度が入っている
    lats, lons = grb_1.latlons()

    #一次元に変換
    # 二次元のままでは描画できない
    # 2次元の格子点を1次元配列にならす
    flat_lats= np.ravel(lats)
    flat_lons= np.ravel(lons)

    pd_rad= {'lons':flat_lons, 'lats':flat_lats}
    pd_Uwind= {'lons':flat_lons, 'lats':flat_lats}
    pd_Vwind= {'lons':flat_lons, 'lats':flat_lats}
    pd_temp= {'lons':flat_lons, 'lats':flat_lats}
    pd_MSL= {'lons':flat_lons, 'lats':flat_lats}

    i,j,k,l,m =0,0,0,0,0

    for grb in grbs:
        print(grb)

        if grb.name == "Downward short-wave radiation flux":
            radiation = grb.values
            flat_radiation = np.ravel(radiation)
            pd_name="radiation FH"+str(i)+"-"+str(i+1)+"W/m2(avg)"
            pd_rad[pd_name]=flat_radiation
            i=i+1

        elif grb.name == "10 metre U wind component":
            Uwind = grb.values
            flat_Uwind = np.ravel(Uwind)
            pd_name="Uwind FH"+str(j)+"m/s"
            pd_Uwind[pd_name]=flat_Uwind
            j=j+1

        elif grb.name == "10 metre V wind component":
            Vwind = grb.values
            flat_Vwind = np.ravel(Vwind)
            pd_name="Vwind FH"+str(k)+"m/s"
            pd_Vwind[pd_name]=flat_Vwind
            k=k+1

        elif grb.name == "Temperature":
            temperature = grb.values
            #華氏を摂氏に変換
            flat_temperature= np.ravel(temperature)
            flat_temperature=flat_temperature-273.115
            pd_name="temperature FH"+str(l)
            pd_temp[pd_name]=flat_temperature
            l=l+1
        
        elif grb.name == "Pressure reduced to MSL":
            MSL = grb.values
            #華氏を摂氏に変換
            flat_MSL= np.ravel(MSL)
            pd_name="Pressure reduced to MSL"+str(m)+" hPa"
            pd_MSL[pd_name]=flat_MSL/100
            m=m+1

    #range adjust
    def range(df):
        df=df[df['lons'] > west_lon]
        df=df[df['lons'] < east_lon]
        df=df[df['lats'] > south_lat]
        df=df[df['lats'] < north_lat]
        return df

    def range_ajust(east_lon,west_lon,north_lat,south_lat,df_all1):
        df=range(df_all1)
        return df

    df_input1=pd.DataFrame(pd_temp)
    df_input2=pd.DataFrame(pd_Uwind)
    df_input3=pd.DataFrame(pd_Vwind)
    df_input4=pd.DataFrame(pd_rad)
    df_input5=pd.DataFrame(pd_MSL)

    df_temp = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input1).reset_index()
    df_Uwind = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input2).reset_index()
    df_Vwind = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input3).reset_index()
    df_rad = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input4).reset_index()
    df_MSL = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input5).reset_index()
    
    new_dir_path="Output_CSV/"+year+"/"+month+"/"+day
    os.makedirs(new_dir_path,exist_ok=True)

    temp_name=new_dir_path+"/"+"FH_"+FH+"_"+"temperature" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
    df_temp.to_csv(temp_name, encoding='utf-8', index=False)

    Uwind_name=new_dir_path+"/"+"FH_"+FH+"_"+"Uwind" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
    df_Uwind.to_csv(Uwind_name, encoding='utf-8', index=False)

    Vwind_name=new_dir_path+"/"+"FH_"+FH+"_"+"Vwind" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
    df_Vwind.to_csv(Vwind_name, encoding='utf-8', index=False)

    rad_name=new_dir_path+"/"+"FH_"+FH+"_"+"radiation" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
    df_rad.to_csv(rad_name, encoding='utf-8', index=False)
    
    MSL_name=new_dir_path+"/"+"FH_"+FH+"_"+"MSL" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
    df_MSL.to_csv(MSL_name, encoding='utf-8', index=False)

    FH="16-33"

    grbs = getgrib(year,month,day,roottime,FH)
    latlons = grbs

    #check parameters in gribfile
    grb_1 = grbs.select(name='Pressure reduced to MSL')[0]

    # lats,lonsは二次元配列で緯度経度が入っている
    lats, lons = grb_1.latlons()

    #一次元に変換
    # 二次元のままでは描画できない
    # 2次元の格子点を1次元配列にならす
    flat_lats= np.ravel(lats)
    flat_lons= np.ravel(lons)

    pd_rad= {'lons':flat_lons, 'lats':flat_lats}
    pd_Uwind= {'lons':flat_lons, 'lats':flat_lats}
    pd_Vwind= {'lons':flat_lons, 'lats':flat_lats}
    pd_temp= {'lons':flat_lons, 'lats':flat_lats}
    pd_MSL= {'lons':flat_lons, 'lats':flat_lats}

    i,j,k,l,m =0,0,0,0,0

    for grb in grbs:
        print(grb)

        if grb.name == "Downward short-wave radiation flux":
            radiation = grb.values
            flat_radiation = np.ravel(radiation)
            pd_name="radiation FH"+str(i)+"-"+str(i+1)+"W/m2(avg)"
            pd_rad[pd_name]=flat_radiation
            i=i+1

        elif grb.name == "10 metre U wind component":
            Uwind = grb.values
            flat_Uwind = np.ravel(Uwind)
            pd_name="Uwind FH"+str(j)+"m/s"
            pd_Uwind[pd_name]=flat_Uwind
            j=j+1

        elif grb.name == "10 metre V wind component":
            Vwind = grb.values
            flat_Vwind = np.ravel(Vwind)
            pd_name="Vwind FH"+str(k)+"m/s"
            pd_Vwind[pd_name]=flat_Vwind
            k=k+1

        elif grb.name == "Temperature":
            temperature = grb.values
            #華氏を摂氏に変換
            flat_temperature= np.ravel(temperature)
            flat_temperature=flat_temperature-273.115
            pd_name="temperature FH"+str(l)
            pd_temp[pd_name]=flat_temperature
            l=l+1
        
        elif grb.name == "Pressure reduced to MSL":
            MSL = grb.values
            #華氏を摂氏に変換
            flat_MSL= np.ravel(MSL)
            pd_name="Pressure reduced to MSL"+str(m)+" hPa"
            pd_MSL[pd_name]=flat_MSL/100
            m=m+1

    #range adjust
    def range(df):
        df=df[df['lons'] > west_lon]
        df=df[df['lons'] < east_lon]
        df=df[df['lats'] > south_lat]
        df=df[df['lats'] < north_lat]
        return df

    def range_ajust(east_lon,west_lon,north_lat,south_lat,df_all1):
        df=range(df_all1)
        return df

    df_input1=pd.DataFrame(pd_temp)
    df_input2=pd.DataFrame(pd_Uwind)
    df_input3=pd.DataFrame(pd_Vwind)
    df_input4=pd.DataFrame(pd_rad)
    df_input5=pd.DataFrame(pd_MSL)

    df_temp = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input1).reset_index()
    df_Uwind = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input2).reset_index()
    df_Vwind = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input3).reset_index()
    df_rad = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input4).reset_index()
    df_MSL = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input5).reset_index()

    print(df_temp)
    
    new_dir_path="Output_CSV/"+year+"/"+month+"/"+day
    os.makedirs(new_dir_path,exist_ok=True)

    temp_name=new_dir_path+"/"+"FH_"+FH+"_"+"temperature" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
    df_temp.to_csv(temp_name, encoding='utf-8', index=False)

    Uwind_name=new_dir_path+"/"+"FH_"+FH+"_"+"Uwind" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
    df_Uwind.to_csv(Uwind_name, encoding='utf-8', index=False)

    Vwind_name=new_dir_path+"/"+"FH_"+FH+"_"+"Vwind" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
    df_Vwind.to_csv(Vwind_name, encoding='utf-8', index=False)

    rad_name=new_dir_path+"/"+"FH_"+FH+"_"+"radiation" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
    df_rad.to_csv(rad_name, encoding='utf-8', index=False)
    
    MSL_name=new_dir_path+"/"+"FH_"+FH+"_"+"MSL" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
    df_MSL.to_csv(MSL_name, encoding='utf-8', index=False)

    FH="34-39"

    grbs = getgrib(year,month,day,roottime,FH)
    latlons = grbs

    #check parameters in gribfile
    grb_1 = grbs.select(name='Pressure reduced to MSL')[0]

    # lats,lonsは二次元配列で緯度経度が入っている
    lats, lons = grb_1.latlons()

    #一次元に変換
    # 二次元のままでは描画できない
    # 2次元の格子点を1次元配列にならす
    flat_lats= np.ravel(lats)
    flat_lons= np.ravel(lons)

    pd_rad= {'lons':flat_lons, 'lats':flat_lats}
    pd_Uwind= {'lons':flat_lons, 'lats':flat_lats}
    pd_Vwind= {'lons':flat_lons, 'lats':flat_lats}
    pd_temp= {'lons':flat_lons, 'lats':flat_lats}
    pd_MSL= {'lons':flat_lons, 'lats':flat_lats}

    i,j,k,l,m =0,0,0,0,0

    for grb in grbs:
        print(grb)

        if grb.name == "Downward short-wave radiation flux":
            radiation = grb.values
            flat_radiation = np.ravel(radiation)
            pd_name="radiation FH"+str(i)+"-"+str(i+1)+"W/m2(avg)"
            pd_rad[pd_name]=flat_radiation
            i=i+1

        elif grb.name == "10 metre U wind component":
            Uwind = grb.values
            flat_Uwind = np.ravel(Uwind)
            pd_name="Uwind FH"+str(j)+"m/s"
            pd_Uwind[pd_name]=flat_Uwind
            j=j+1

        elif grb.name == "10 metre V wind component":
            Vwind = grb.values
            flat_Vwind = np.ravel(Vwind)
            pd_name="Vwind FH"+str(k)+"m/s"
            pd_Vwind[pd_name]=flat_Vwind
            k=k+1

        elif grb.name == "Temperature":
            temperature = grb.values
            #華氏を摂氏に変換
            flat_temperature= np.ravel(temperature)
            flat_temperature=flat_temperature-273.115
            pd_name="temperature FH"+str(l)
            pd_temp[pd_name]=flat_temperature
            l=l+1
        
        elif grb.name == "Pressure reduced to MSL":
            MSL = grb.values
            #華氏を摂氏に変換
            flat_MSL= np.ravel(MSL)
            pd_name="Pressure reduced to MSL"+str(m)+" hPa"
            pd_MSL[pd_name]=flat_MSL/100
            m=m+1

    #range adjust
    def range(df):
        df=df[df['lons'] > west_lon]
        df=df[df['lons'] < east_lon]
        df=df[df['lats'] > south_lat]
        df=df[df['lats'] < north_lat]
        return df

    def range_ajust(east_lon,west_lon,north_lat,south_lat,df_all1):
        df=range(df_all1)
        return df

    df_input1=pd.DataFrame(pd_temp)
    df_input2=pd.DataFrame(pd_Uwind)
    df_input3=pd.DataFrame(pd_Vwind)
    df_input4=pd.DataFrame(pd_rad)
    df_input5=pd.DataFrame(pd_MSL)

    df_temp = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input1).reset_index()
    df_Uwind = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input2).reset_index()
    df_Vwind = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input3).reset_index()
    df_rad = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input4).reset_index()
    df_MSL = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input5).reset_index()
    
    new_dir_path="Output_CSV/"+year+"/"+month+"/"+day
    os.makedirs(new_dir_path,exist_ok=True)

    temp_name=new_dir_path+"/"+"FH_"+FH+"_"+"temperature" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
    df_temp.to_csv(temp_name, encoding='utf-8', index=False)

    Uwind_name=new_dir_path+"/"+"FH_"+FH+"_"+"Uwind" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
    df_Uwind.to_csv(Uwind_name, encoding='utf-8', index=False)

    Vwind_name=new_dir_path+"/"+"FH_"+FH+"_"+"Vwind" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
    df_Vwind.to_csv(Vwind_name, encoding='utf-8', index=False)

    rad_name=new_dir_path+"/"+"FH_"+FH+"_"+"radiation" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
    df_rad.to_csv(rad_name, encoding='utf-8', index=False)
    
    MSL_name=new_dir_path+"/"+"FH_"+FH+"_"+"MSL" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
    df_MSL.to_csv(MSL_name, encoding='utf-8', index=False)

    FH="40-51"

    grbs = getgrib(year,month,day,roottime,FH)
    latlons = grbs

    #check parameters in gribfile
    grb_1 = grbs.select(name='Pressure reduced to MSL')[0]

    # lats,lonsは二次元配列で緯度経度が入っている
    lats, lons = grb_1.latlons()

    #一次元に変換
    # 二次元のままでは描画できない
    # 2次元の格子点を1次元配列にならす
    flat_lats= np.ravel(lats)
    flat_lons= np.ravel(lons)

    pd_rad= {'lons':flat_lons, 'lats':flat_lats}
    pd_Uwind= {'lons':flat_lons, 'lats':flat_lats}
    pd_Vwind= {'lons':flat_lons, 'lats':flat_lats}
    pd_temp= {'lons':flat_lons, 'lats':flat_lats}
    pd_MSL= {'lons':flat_lons, 'lats':flat_lats}

    i,j,k,l,m =0,0,0,0,0

    for grb in grbs:
        print(grb)

        if grb.name == "Downward short-wave radiation flux":
            radiation = grb.values
            flat_radiation = np.ravel(radiation)
            pd_name="radiation FH"+str(i)+"-"+str(i+1)+"W/m2(avg)"
            pd_rad[pd_name]=flat_radiation
            i=i+1

        elif grb.name == "10 metre U wind component":
            Uwind = grb.values
            flat_Uwind = np.ravel(Uwind)
            pd_name="Uwind FH"+str(j)+"m/s"
            pd_Uwind[pd_name]=flat_Uwind
            j=j+1

        elif grb.name == "10 metre V wind component":
            Vwind = grb.values
            flat_Vwind = np.ravel(Vwind)
            pd_name="Vwind FH"+str(k)+"m/s"
            pd_Vwind[pd_name]=flat_Vwind
            k=k+1

        elif grb.name == "Temperature":
            temperature = grb.values
            #華氏を摂氏に変換
            flat_temperature= np.ravel(temperature)
            flat_temperature=flat_temperature-273.115
            pd_name="temperature FH"+str(l)
            pd_temp[pd_name]=flat_temperature
            l=l+1
        
        elif grb.name == "Pressure reduced to MSL":
            MSL = grb.values
            #華氏を摂氏に変換
            flat_MSL= np.ravel(MSL)
            pd_name="Pressure reduced to MSL"+str(m)+" hPa"
            pd_MSL[pd_name]=flat_MSL/100
            m=m+1

    #range adjust
    def range(df):
        df=df[df['lons'] > west_lon]
        df=df[df['lons'] < east_lon]
        df=df[df['lats'] > south_lat]
        df=df[df['lats'] < north_lat]
        return df

    def range_ajust(east_lon,west_lon,north_lat,south_lat,df_all1):
        df=range(df_all1)
        return df

    df_input1=pd.DataFrame(pd_temp)
    df_input2=pd.DataFrame(pd_Uwind)
    df_input3=pd.DataFrame(pd_Vwind)
    df_input4=pd.DataFrame(pd_rad)
    df_input5=pd.DataFrame(pd_MSL)

    df_temp = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input1).reset_index()
    df_Uwind = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input2).reset_index()
    df_Vwind = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input3).reset_index()
    df_rad = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input4).reset_index()
    df_MSL = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input5).reset_index()
    
    new_dir_path="Output_CSV/"+year+"/"+month+"/"+day
    os.makedirs(new_dir_path,exist_ok=True)

    temp_name=new_dir_path+"/"+"FH_"+FH+"_"+"temperature" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
    df_temp.to_csv(temp_name, encoding='utf-8', index=False)

    Uwind_name=new_dir_path+"/"+"FH_"+FH+"_"+"Uwind" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
    df_Uwind.to_csv(Uwind_name, encoding='utf-8', index=False)

    Vwind_name=new_dir_path+"/"+"FH_"+FH+"_"+"Vwind" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
    df_Vwind.to_csv(Vwind_name, encoding='utf-8', index=False)

    rad_name=new_dir_path+"/"+"FH_"+FH+"_"+"radiation" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
    df_rad.to_csv(rad_name, encoding='utf-8', index=False)
    
    MSL_name=new_dir_path+"/"+"FH_"+FH+"_"+"MSL" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
    df_MSL.to_csv(MSL_name, encoding='utf-8', index=False)

    day=int(day)
    day2=int(day2)
    
    if day == day2:
        print('break!')
        break
    else:
        day=day+1
        
    day=str(day)
    day2=str(day2)

1:Pressure reduced to MSL:Pa (instant):regular_ll:meanSea:level 0:fcst time 0 hrs:from 201906150000
2:Surface pressure:Pa (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 201906150000
3:10 metre U wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 0 hrs:from 201906150000
4:10 metre V wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 0 hrs:from 201906150000
5:Temperature:K (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 0 hrs:from 201906150000
6:Relative humidity:% (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 0 hrs:from 201906150000
7:Low cloud cover:% (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 201906150000
8:Medium cloud cover:% (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 201906150000
9:High cloud cover:% (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 201906150000
10:Total cloud cover:% (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 2

107:Pressure reduced to MSL:Pa (instant):regular_ll:meanSea:level 0:fcst time 9 hrs:from 201906150000
108:Surface pressure:Pa (instant):regular_ll:surface:level 0:fcst time 9 hrs:from 201906150000
109:10 metre U wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 9 hrs:from 201906150000
110:10 metre V wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 9 hrs:from 201906150000
111:Temperature:K (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 9 hrs:from 201906150000
112:Relative humidity:% (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 9 hrs:from 201906150000
113:Low cloud cover:% (instant):regular_ll:surface:level 0:fcst time 9 hrs:from 201906150000
114:Medium cloud cover:% (instant):regular_ll:surface:level 0:fcst time 9 hrs:from 201906150000
115:High cloud cover:% (instant):regular_ll:surface:level 0:fcst time 9 hrs:from 201906150000
116:Total cloud cover:% (instant):regular_ll:surface:level 0:fcs

1:Pressure reduced to MSL:Pa (instant):regular_ll:meanSea:level 0:fcst time 16 hrs:from 201906150000
2:Surface pressure:Pa (instant):regular_ll:surface:level 0:fcst time 16 hrs:from 201906150000
3:10 metre U wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 16 hrs:from 201906150000
4:10 metre V wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 16 hrs:from 201906150000
5:Temperature:K (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 16 hrs:from 201906150000
6:Relative humidity:% (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 16 hrs:from 201906150000
7:Low cloud cover:% (instant):regular_ll:surface:level 0:fcst time 16 hrs:from 201906150000
8:Medium cloud cover:% (instant):regular_ll:surface:level 0:fcst time 16 hrs:from 201906150000
9:High cloud cover:% (instant):regular_ll:surface:level 0:fcst time 16 hrs:from 201906150000
10:Total cloud cover:% (instant):regular_ll:surface:level 0:fcst time 16 

121:Pressure reduced to MSL:Pa (instant):regular_ll:meanSea:level 0:fcst time 26 hrs:from 201906150000
122:Surface pressure:Pa (instant):regular_ll:surface:level 0:fcst time 26 hrs:from 201906150000
123:10 metre U wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 26 hrs:from 201906150000
124:10 metre V wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 26 hrs:from 201906150000
125:Temperature:K (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 26 hrs:from 201906150000
126:Relative humidity:% (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 26 hrs:from 201906150000
127:Low cloud cover:% (instant):regular_ll:surface:level 0:fcst time 26 hrs:from 201906150000
128:Medium cloud cover:% (instant):regular_ll:surface:level 0:fcst time 26 hrs:from 201906150000
129:High cloud cover:% (instant):regular_ll:surface:level 0:fcst time 26 hrs:from 201906150000
130:Total cloud cover:% (instant):regular_ll:surface:le

    index     lons  lats  temperature FH0  temperature FH1  temperature FH2  \
0  134846  130.375  33.6        20.579763        20.356863        20.210043   

   temperature FH3  temperature FH4  temperature FH5  temperature FH6  ...  \
0        20.317983        20.225118        19.926534        19.987722  ...   

   temperature FH8  temperature FH9  temperature FH10  temperature FH11  \
0        20.594534        20.888693         21.028188         21.131521   

   temperature FH12  temperature FH13  temperature FH14  temperature FH15  \
0         21.179281         21.201742         21.112448         21.081014   

   temperature FH16  temperature FH17  
0         20.960989         20.715292  

[1 rows x 21 columns]
1:Pressure reduced to MSL:Pa (instant):regular_ll:meanSea:level 0:fcst time 34 hrs:from 201906150000
2:Surface pressure:Pa (instant):regular_ll:surface:level 0:fcst time 34 hrs:from 201906150000
3:10 metre U wind component:m s**-1 (instant):regular_ll:heightAboveGround:level

...

179:Pressure reduced to MSL:Pa (instant):regular_ll:meanSea:level 0:fcst time 15 hrs:from 201906230000
180:Surface pressure:Pa (instant):regular_ll:surface:level 0:fcst time 15 hrs:from 201906230000
181:10 metre U wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 15 hrs:from 201906230000
182:10 metre V wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 15 hrs:from 201906230000
183:Temperature:K (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 15 hrs:from 201906230000
184:Relative humidity:% (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 15 hrs:from 201906230000
185:Low cloud cover:% (instant):regular_ll:surface:level 0:fcst time 15 hrs:from 201906230000
186:Medium cloud cover:% (instant):regular_ll:surface:level 0:fcst time 15 hrs:from 201906230000
187:High cloud cover:% (instant):regular_ll:surface:level 0:fcst time 15 hrs:from 201906230000
188:Total cloud cover:% (instant):regular_ll:surface:le

122:Surface pressure:Pa (instant):regular_ll:surface:level 0:fcst time 26 hrs:from 201906230000
123:10 metre U wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 26 hrs:from 201906230000
124:10 metre V wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 26 hrs:from 201906230000
125:Temperature:K (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 26 hrs:from 201906230000
126:Relative humidity:% (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 26 hrs:from 201906230000
127:Low cloud cover:% (instant):regular_ll:surface:level 0:fcst time 26 hrs:from 201906230000
128:Medium cloud cover:% (instant):regular_ll:surface:level 0:fcst time 26 hrs:from 201906230000
129:High cloud cover:% (instant):regular_ll:surface:level 0:fcst time 26 hrs:from 201906230000
130:Total cloud cover:% (instant):regular_ll:surface:level 0:fcst time 26 hrs:from 201906230000
131:Total precipitation:kg m-2 (accum):regular_ll:surface:leve

    index     lons  lats  temperature FH0  temperature FH1  temperature FH2  \
0  134846  130.375  33.6        20.300894        20.541708        20.686086   

   temperature FH3  temperature FH4  temperature FH5  temperature FH6  ...  \
0        20.836263        20.968191         20.78594        20.936971  ...   

   temperature FH8  temperature FH9  temperature FH10  temperature FH11  \
0        21.574911        21.798849         21.871725         21.949911   

   temperature FH12  temperature FH13  temperature FH14  temperature FH15  \
0         21.966482         21.979299         22.001699         21.937826   

   temperature FH16  temperature FH17  
0         21.779928          21.56014  

[1 rows x 21 columns]
1:Pressure reduced to MSL:Pa (instant):regular_ll:meanSea:level 0:fcst time 34 hrs:from 201906230000
2:Surface pressure:Pa (instant):regular_ll:surface:level 0:fcst time 34 hrs:from 201906230000
3:10 metre U wind component:m s**-1 (instant):regular_ll:heightAboveGround:level

1:Pressure reduced to MSL:Pa (instant):regular_ll:meanSea:level 0:fcst time 40 hrs:from 201906230000
2:Surface pressure:Pa (instant):regular_ll:surface:level 0:fcst time 40 hrs:from 201906230000
3:10 metre U wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 40 hrs:from 201906230000
4:10 metre V wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 40 hrs:from 201906230000
5:Temperature:K (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 40 hrs:from 201906230000
6:Relative humidity:% (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 40 hrs:from 201906230000
7:Low cloud cover:% (instant):regular_ll:surface:level 0:fcst time 40 hrs:from 201906230000
8:Medium cloud cover:% (instant):regular_ll:surface:level 0:fcst time 40 hrs:from 201906230000
9:High cloud cover:% (instant):regular_ll:surface:level 0:fcst time 40 hrs:from 201906230000
10:Total cloud cover:% (instant):regular_ll:surface:level 0:fcst time 40 

121:Pressure reduced to MSL:Pa (instant):regular_ll:meanSea:level 0:fcst time 50 hrs:from 201906230000
122:Surface pressure:Pa (instant):regular_ll:surface:level 0:fcst time 50 hrs:from 201906230000
123:10 metre U wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 50 hrs:from 201906230000
124:10 metre V wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 50 hrs:from 201906230000
125:Temperature:K (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 50 hrs:from 201906230000
126:Relative humidity:% (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 50 hrs:from 201906230000
127:Low cloud cover:% (instant):regular_ll:surface:level 0:fcst time 50 hrs:from 201906230000
128:Medium cloud cover:% (instant):regular_ll:surface:level 0:fcst time 50 hrs:from 201906230000
129:High cloud cover:% (instant):regular_ll:surface:level 0:fcst time 50 hrs:from 201906230000
130:Total cloud cover:% (instant):regular_ll:surface:le

In [3]:
elapsed_time = time.time() - start

print("elapsed_time:{0}".format(elapsed_time))

elapsed_time:47.528249740600586


In [4]:
#緯度経度の範囲を選択
east_lon=130.400
west_lon= 130.370
north_lat=33.61
south_lat=33.59
df_MSL = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input5).reset_index()
df_MSL

,index,lons,lats,Pressure reduced to MSL0 hPa,Pressure reduced to MSL1 hPa,Pressure reduced to MSL2 hPa,Pressure reduced to MSL3 hPa,Pressure reduced to MSL4 hPa,Pressure reduced to MSL5 hPa,Pressure reduced to MSL6 hPa,Pressure reduced to MSL7 hPa,Pressure reduced to MSL8 hPa,Pressure reduced to MSL9 hPa,Pressure reduced to MSL10 hPa,Pressure reduced to MSL11 hPa
0,134846,130.375,33.6,1014.381409,1014.196899,1013.922852,1014.150208,1014.634827,1015.044556,1015.351562,1015.378601,1015.433594,1015.346497,1015.117432,1014.550842
